# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [16]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto natales,-51.7236,-72.4875,9.25,76,75,7.20,CL,1738844080
1,1,papatowai,-46.5619,169.4708,9.14,89,17,2.90,NZ,1738844081
2,2,hasaki,35.7333,140.8333,6.56,31,21,3.92,JP,1738844082
3,3,san patricio,28.0170,-97.5169,20.97,94,31,4.99,US,1738844083
4,4,albany,42.6001,-73.9662,-6.88,80,100,4.44,US,1738844084


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Configure the map plot

map_1 = city_data_df.hvplot.points(
    'Lng',          # Column for longitude
    'Lat',           # Column for latitude
    geo=True,             # Enable geographic plotting
    tiles='OSM',          # Use OpenStreetMap tiles
    size='Humidity',      # Size of points based on humidity
    scale=1,            # Scale factor for size
    frame_width=800,      # Width of the plot
    frame_height=600,     # Height of the plot
    color='City',         # Color points by city
)

# Display the map
map_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [23]:
# Narrow down cities that fit criteria and drop any results with null values

my_climate = city_data_df.loc[(city_data_df["Max Temp"] > 20) & (city_data_df["Humidity"] <80) & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
cleaned_my_climate = my_climate.dropna(how='any')


# Display sample data
cleaned_my_climate.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
28,28,arraial do cabo,-22.9661,-42.0278,28.92,61,0,7.72,BR,1738843930
55,55,didwana,27.4000,74.5667,21.50,12,0,3.26,IN,1738844143
119,119,margaret river,-33.9500,115.0667,20.56,67,0,8.35,AU,1738844216
169,169,howrah,22.5892,88.3103,26.00,47,0,3.60,IN,1738844278
183,183,sanya,18.2431,109.5050,22.09,73,0,6.00,CN,1738844296


### Step 3: Create a new DataFrame called `hotel_df`.

In [35]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = cleaned_my_climate.copy(deep=True)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""
del hotel_df['City_ID']
del hotel_df['Max Temp']
del hotel_df['Cloudiness']
del hotel_df['Wind Speed']
del hotel_df['Date']

# Display sample data
hotel_df.head()

,City,Lat,Lng,Humidity,Country,Hotel Name
28,arraial do cabo,-22.9661,-42.0278,61,BR,
55,didwana,27.4000,74.5667,12,IN,
119,margaret river,-33.9500,115.0667,67,AU,
169,howrah,22.5892,88.3103,47,IN,
183,sanya,18.2431,109.5050,73,CN,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [43]:
# Set parameters to search for a hotel
radius = 100000
params = {
    "categories": "accommodation.hotel",  # Specify the category for hotels
    "apiKey": geoapify_key,  # Your Geoapify API key
    "limit": 10  # Limit the number of results returned
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]  # Assuming the DataFrame has a 'Lat' column
    longitude = row["Lng"]  # Assuming the DataFrame has a 'Lon' column

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"  # Define the search area
    params["bias"] = f"proximity:{longitude},{latitude}"  # Bias the search towards the location

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params = params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    #Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
arraial do cabo - nearest hotel: No hotel found
didwana - nearest hotel: No hotel found
margaret river - nearest hotel: Margaret River Hotel
howrah - nearest hotel: Sun India Guest House
sanya - nearest hotel: No hotel found
adrar - nearest hotel: No hotel found
viedma - nearest hotel: Níjar
rosso - nearest hotel: نزل صانتر روصو
taoudenni - nearest hotel: No hotel found
stawell - nearest hotel: National Hotel
itacarambi - nearest hotel: No hotel found
illizi - nearest hotel: بوناقة
sofo-birnin-gwari - nearest hotel: No hotel found
ma'rib - nearest hotel: No hotel found
vihari - nearest hotel: Tanveer Aslam
tionk essil - nearest hotel: Campement Edonia
akjoujt - nearest hotel: Hotel Fatima فندق فاطمة
villa gesell - nearest hotel: Hotel El Faro
zabid - nearest hotel: منزل علاء جحاف
mahina - nearest hotel: Motu Martin
kolda - nearest hotel: le firdou
katghora - nearest hotel: No hotel found
illapel - nearest hotel: Domingo Ortiz de Rosas


,City,Lat,Lng,Humidity,Country,Hotel Name
28,arraial do cabo,-22.9661,-42.0278,61,BR,No hotel found
55,didwana,27.4000,74.5667,12,IN,No hotel found
119,margaret river,-33.9500,115.0667,67,AU,Margaret River Hotel
169,howrah,22.5892,88.3103,47,IN,Sun India Guest House
183,sanya,18.2431,109.5050,73,CN,No hotel found
195,adrar,20.5022,-10.0711,14,MR,No hotel found
205,viedma,-40.8135,-62.9967,30,AR,Níjar
217,rosso,16.4986,-15.8153,9,SN,نزل صانتر روصو
238,taoudenni,22.6783,-3.9836,17,ML,No hotel found
282,stawell,-37.0500,142.7667,49,AU,National Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [49]:
%%capture --no-display

# Configure the map plot
map_2 = hotel_df.hvplot.points(
    'Lng',          # Column for longitude
    'Lat',           # Column for latitude
    geo=True,             # Enable geographic plotting
    tiles='OSM',          # Use OpenStreetMap tiles
    size='Humidity',      # Size of points based on humidity
    scale=1,            # Scale factor for size
    frame_width=800,      # Width of the plot
    frame_height=600,     # Height of the plot
    color='City',         # Color points by city
    hover_cols = ['Hotel Name', 'Country']  # Hotel name and country as hover
)

# Display the map
map_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)